In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',None)
import json
from geopy.geocoders import Nominatim
import requests
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium


In [2]:
# the data was downloaded from the link https://geo.nyu.edu/catalog/nyu_2451_34572 

path = 'file path'
with open(path) as json_data:
    newyork_data = json.load(json_data)

In [9]:
neighborhoods_data = newyork_data['features']
neighborhoods_data[0]

#define the dataframe columns to convert it into the pandas dataframe
column_names = ['Borough','Neighborhood','Latitude','Longitude']

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

# looping through data and fill the dataframe one row at a time:
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough']
    neighborhood_name = data['properties']['name']
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                         'Neighborhood': neighborhood_name,
                                         'Latitude': neighborhood_lat,
                                         'Longitude': neighborhood_lon}, ignore_index=True)

print ('The dataframe has {} borughs and {} neighborhoods.'.format(
      len(neighborhoods['Borough'].unique()), neighborhoods.shape[0]))

The dataframe has 5 borughs and 306 neighborhoods.


In [21]:
# get info on Manhattan Borough's Neighborhoods and its geolocations
manhattan_data = neighborhoods[neighborhoods['Borough']=='Manhattan'].reset_index(drop=True)
print(manhattan_data.head())

# getting the geographical coordinate of Manhattan
address = 'Manhattan, NY'
geolocator = Nominatim(user_agent = "ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print("The geographical coordinates of Manhattan are {}, {}".format(latitude, longitude))

# creating a map of Manhattan
map_manhattan = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(manhattan_data['Latitude'], manhattan_data['Longitude'], manhattan_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_manhattan)  
    
map_manhattan


     Borough        Neighborhood   Latitude  Longitude
0  Manhattan         Marble Hill  40.876551 -73.910660
1  Manhattan           Chinatown  40.715618 -73.994279
2  Manhattan  Washington Heights  40.851903 -73.936900
3  Manhattan              Inwood  40.867684 -73.921210
4  Manhattan    Hamilton Heights  40.823604 -73.949688
The geographical coordinates of Manhattan are 40.7896239, -73.9598939


In [22]:
# Define FourSquare Credentials and Version
CLIENT_ID = '' # your Foursquare ID
CLIENT_SECRET = '' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: FDXDMA1KQEDDZLQYZCRPMLICL2CJWYCX5QRMMB2XMVDP44KS
CLIENT_SECRET:YJSYMICCRB0A2VTTX4EWWDOJCKZ1MD2TQ0OE3G5P4ETQFIIC


In [24]:
# get neighborhoods latitude and longitude values of Manhattan
neighborhood_latitude = manhattan_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = manhattan_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = manhattan_data.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Marble Hill are 40.87655077879964, -73.91065965862981.


In [25]:
# create a GET request and URL.
LIMIT = 500
radius = 2000
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        neighborhood_latitude,
        neighborhood_longitude,
        radius,
        LIMIT
        )
url

'https://api.foursquare.com/v2/venues/explore?&client_id=FDXDMA1KQEDDZLQYZCRPMLICL2CJWYCX5QRMMB2XMVDP44KS&client_secret=YJSYMICCRB0A2VTTX4EWWDOJCKZ1MD2TQ0OE3G5P4ETQFIIC&v=20180605&ll=40.87655077879964,-73.91065965862981&radius=2000&limit=500'

In [31]:
# send the get requests and examine the results

results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5ee86f7275a94d3ebe7d24cd'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': '$-$$$$', 'key': 'price'},
    {'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Marble Hill',
  'headerFullLocation': 'Marble Hill, New York',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 190,
  'suggestedBounds': {'ne': {'lat': 40.894550796799656,
    'lng': -73.88689838082279},
   'sw': {'lat': 40.85855076079962, 'lng': -73.93442093643684}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4baf59e8f964a520a6f93be3',
       'name': 'Bikram Yoga',
       'location': {'address': '5500 Broadway',
        'crossStreet': '230th Street',
        'lat': 40.876843690797934,
        'lng': -73.90620

In [35]:
# function that extracts the category of the venue

def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [37]:
# clean the json data and structure it into pandas dataframe

venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()
print('{} venues were returned by FourSquare.'.format(nearby_venues.shape[0]))

100 venues were returned by FourSquare.


/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  """


In [26]:
# exploring the neighborhoods of Manhattan
def getNearbyVenues(names, latitudes, longitudes, radius=2000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)


In [38]:
# create a new dataframe called manhattan_venues.
manhattan_venues = getNearbyVenues(names=manhattan_data['Neighborhood'],
                                   latitudes=manhattan_data['Latitude'],
                                   longitudes=manhattan_data['Longitude']
                                  )
print(manhattan_venues.shape)
manhattan_venues.head()

Marble Hill
Chinatown
Washington Heights
Inwood
Hamilton Heights
Manhattanville
Central Harlem
East Harlem
Upper East Side
Yorkville
Lenox Hill
Roosevelt Island
Upper West Side
Lincoln Square
Clinton
Midtown
Murray Hill
Chelsea
Greenwich Village
East Village
Lower East Side
Tribeca
Little Italy
Soho
West Village
Manhattan Valley
Morningside Heights
Gramercy
Battery Park City
Financial District
Carnegie Hill
Noho
Civic Center
Midtown South
Sutton Place
Turtle Bay
Tudor City
Stuyvesant Town
Flatiron
Hudson Yards
(4000, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Marble Hill,40.876551,-73.91066,Bikram Yoga,40.876844,-73.906204,Yoga Studio
1,Marble Hill,40.876551,-73.91066,Sam's Pizza,40.879435,-73.905859,Pizza Place
2,Marble Hill,40.876551,-73.91066,Tibbett Diner,40.880404,-73.908937,Diner
3,Marble Hill,40.876551,-73.91066,Arturo's,40.874412,-73.910271,Pizza Place
4,Marble Hill,40.876551,-73.91066,El Malecon,40.879338,-73.904457,Caribbean Restaurant


In [40]:
manhattan_venues.groupby('Neighborhood').count()
print('There are {} unique categories.'.format(len(manhattan_venues['Venue Category'].unique())))

There are 250 unique categories.


In [42]:
# Analyze Each Neighborhood

# one hot encoding
manhattan_onehot = pd.get_dummies(manhattan_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
manhattan_onehot['Neighborhood'] = manhattan_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [manhattan_onehot.columns[-1]] + list(manhattan_onehot.columns[:-1])
manhattan_onehot = manhattan_onehot[fixed_columns]

manhattan_onehot.head()
manhattan_onehot.shape

(4000, 251)

In [46]:
# Group rows by neighborhood and the mean of the frequency of occurrence of each category

manhattan_grouped = manhattan_onehot.groupby('Neighborhood').mean().reset_index()
manhattan_grouped

,Neighborhood,Adult Boutique,African Restaurant,American Restaurant,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Basketball Stadium,Beach,Beer Bar,Beer Garden,Beer Store,Bike Shop,Bike Trail,Bistro,Bookstore,Boutique,Boxing Gym,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Building,Burger Joint,Butcher,Café,Cambodian Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Castle,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Circus,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Arts Building,College Theater,Comedy Club,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Cosmetics Shop,Cuban Restaurant,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Dog Run,Donut Shop,Dumpling Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farm,Farmers Market,Field,Filipino Restaurant,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden,Garden Center,Gastropub,General Entertainment,German Restaurant,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,Historic Site,History Museum,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Korean Restaurant,Latin American Restaurant,Laundry Service,Lebanese Restaurant,Library,Lighthouse,Liquor Store,Lounge,Market,Martial Arts Dojo,Mediterranean Restaurant,Memorial Site,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Store,Music Venue,Nail Salon,Nature Preserve,New American Restaurant,Newsstand,Noodle House,Opera House,Organic Grocery,Outdoor Sculpture,Outdoors & Recreation,Paper / Office Supplies Store,Park,Pedestrian Plaza,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Café,Pet Store,Pharmacy,Photography Studio,Pie Shop,Pier,Pilates Studio,Pizza Place,Planetarium,Playground,Plaza,Poke Place,Pool,Pub,Public Art,Ramen Restaurant,Recreation Center,Reservoir,Resort,Restaurant,Rock Club,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,Science Museum,Sculpture Garden,Seafood Restaurant,Shanghai Restaurant,Shoe Store,Shopping Mall,Smoke Shop,Snack Place,Soup Place,Southern / Soul Food Restaurant,Souvenir Shop,Souvlaki Shop,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,State / Provincial Park,Stationery Store,Steakhouse,Street Art,Supermarket,Sushi Restaurant,Szechuan Restaurant,TV Station,Taco Place,Tailor Shop,Tapas Restaurant,Tea Room,Tech Startup,Tennis Court,Thai Restaurant,Theater,Theme Park Ride / Attraction,Toy / Game Store,Track,Trail,Train Station,Tram Station,Turkish Restaurant,Udon Restaurant,Vegetarian / Vegan Restaurant,Venezuelan Restaurant,Veterinarian,Video Game Store,Vietnamese Restaurant,Volleyball Court,Warehouse Store,Waterfront,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Battery Park City,0.00,0.00,0.00,0.01,0.00,0.00,0.00,0.01,0.00,0.00,0.01,0.01,0.01,0.00,0.00,0.00,0.00,0.01,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.01,0.00,0.00,0.00,0.01,0.02,0.00,0.01,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.02,0.07,0.00,0.00,0.00,0.00,0.01,0.00,0.00,0.00,0.00,0.00,0.01,0.01,0.00,0.00,0.00,0.00,0.01,0.00,0.00,0.01,0.00,0.00,0.00,0.00,0.01,0.00,0.01,0.00,0.00,

In [47]:
# Print each neighborhood along with the top 5 most common venues

manhattan_grouped.shape
num_top_venues = 10

for hood in manhattan_grouped['Neighborhood']:
    print('------'+hood+'-------')
    temp = manhattan_grouped[manhattan_grouped['Neighborhood']==hood].T.reset_index()
    temp.columns = ['venue', 'freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq':2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')
    

------Battery Park City-------
                  venue  freq
0                  Park  0.12
1           Coffee Shop  0.07
2                 Hotel  0.04
3                   Gym  0.04
4  Gym / Fitness Center  0.03
5    Italian Restaurant  0.03
6         Memorial Site  0.03
7                 Plaza  0.03
8        Scenic Lookout  0.02
9           Pizza Place  0.02


------Carnegie Hill-------
                 venue  freq
0   Italian Restaurant  0.07
1          Coffee Shop  0.06
2              Exhibit  0.05
3               Bakery  0.05
4                 Park  0.05
5           Art Museum  0.04
6  American Restaurant  0.03
7            Wine Shop  0.03
8                 Café  0.03
9          Pizza Place  0.03


------Central Harlem-------
                             venue  freq
0                      Coffee Shop  0.07
1                              Bar  0.06
2                             Park  0.05
3                      Yoga Studio  0.04
4  Southern / Soul Food Restaurant  0.04
5              

9           Coffee Shop  0.02


------Morningside Heights-------
                 venue  freq
0                 Park  0.10
1          Coffee Shop  0.06
2   Italian Restaurant  0.05
3  American Restaurant  0.04
4          Pizza Place  0.03
5        Grocery Store  0.03
6           Playground  0.03
7   Seafood Restaurant  0.03
8           Bagel Shop  0.02
9              Dog Run  0.02


------Murray Hill-------
                  venue  freq
0               Theater  0.08
1                 Hotel  0.06
2   American Restaurant  0.04
3          Gourmet Shop  0.04
4        Sandwich Place  0.03
5          Burger Joint  0.03
6                  Park  0.03
7  Gym / Fitness Center  0.03
8            Boxing Gym  0.03
9          Concert Hall  0.02


------Noho-------
             venue  freq
0           Bakery  0.03
1   Sandwich Place  0.03
2   Ice Cream Shop  0.03
3         Wine Bar  0.03
4      Coffee Shop  0.03
5             Café  0.03
6  Thai Restaurant  0.03
7            Hotel  0.02
8             

In [49]:
# write a function to sort the venues in descending order

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

# create a new dataframe displaying the top 10 venues for each neighborhood

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = manhattan_grouped['Neighborhood']

for ind in np.arange(manhattan_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(manhattan_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Battery Park City,Park,Coffee Shop,Gym,Hotel,Italian Restaurant,Memorial Site,Plaza,Gym / Fitness Center,Steakhouse,Burger Joint
1,Carnegie Hill,Italian Restaurant,Coffee Shop,Bakery,Exhibit,Park,Art Museum,Pizza Place,Café,Gym,Bar
2,Central Harlem,Coffee Shop,Bar,Park,Cocktail Bar,Southern / Soul Food Restaurant,Yoga Studio,American Restaurant,Theater,Wine Shop,Burger Joint
3,Chelsea,Park,Gym / Fitness Center,Bakery,Yoga Studio,Hotel,Coffee Shop,Jazz Club,Gourmet Shop,Beer Bar,New American Restaurant
4,Chinatown,Hotel,Sandwich Place,Pizza Place,Bakery,Thai Restaurant,Italian Restaurant,Spa,Yoga Studio,Gym / Fitness Center,Sushi Restaurant
5,Civic Center,Park,Coffee Shop,Hotel,Italian Restaurant,Bakery,Memorial Site,Café,Dance Studio,Falafel Restaurant,Thai Restaurant
6,Clinton,Theater,Park,Coffee Shop,Mediterranean Restaurant,Pizza Place,Gym / Fitness Center,Gym,Concert Hall,Performing Arts Venue,Indie Theater
7,East Harlem,Park,Café,Coffee Shop,Pizza Place,Gym,Plaza,Italian Restaurant,Fountain,Cocktail Bar,Thai Restaurant
8,East Village,Pizza Place,Ice Cream Shop,Gourmet Shop,Bakery,Café,Park,Wine Shop,Wine Bar,Speakeasy,Hotel
9,Financial District,Park,Coffee Shop,Gym / Fitness Center,Gym,Pizza Place,Plaza,Cocktail Bar,Hotel,Memorial Site,Burger Joint


# Cluster Neighborhoods

In [50]:
# run k-means to cluster the neighborhoods into 5 clusters.

kclusters = 5

manhattan_grouped_clustering = manhattan_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(manhattan_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 1, 3, 1, 1, 2, 0, 2, 1, 2], dtype=int32)

In [51]:
# create a dataframe that includes cluster as well as the top 10 venues

# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

manhattan_merged = manhattan_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
manhattan_merged = manhattan_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

manhattan_merged.head() # check the last columns!

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Manhattan,Marble Hill,40.876551,-73.910660,4,Mexican Restaurant,Pizza Place,Park,Diner,Bar,Café,Coffee Shop,Deli / Bodega,Spanish Restaurant,Bakery
1,Manhattan,Chinatown,40.715618,-73.994279,1,Hotel,Sandwich Place,Pizza Place,Bakery,Thai Restaurant,Italian Restaurant,Spa,Yoga Studio,Gym / Fitness Center,Sushi Restaurant
2,Manhattan,Washington Heights,40.851903,-73.936900,4,Park,Pizza Place,Latin American Restaurant,Mexican Restaurant,Café,Spanish Restaurant,Bakery,Bar,Wine Shop,Wine Bar
3,Manhattan,Inwood,40.867684,-73.921210,4,Pizza Place,Mexican Restaurant,Café,Park,Latin American Restaurant,Wine Bar,Deli / Bodega,Restaurant,Spanish Restaurant,Scenic Lookout
4,Manhattan,Hamilton Heights,40.823604,-73.949688,3,Coffee Shop,Park,Bar,Italian Restaurant,Yoga Studio,French Restaurant,Cocktail Bar,Tennis Court,Jazz Club,Juice Bar


In [52]:
# Visualize the clusters

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(manhattan_merged['Latitude'], manhattan_merged['Longitude'], manhattan_merged['Neighborhood'], manhattan_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

# Examine Clusters

### Cluster 1

In [53]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 0, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
13,Lincoln Square,Theater,Park,Concert Hall,Gym,Bakery,Jazz Club,Spa,Thai Restaurant,Plaza,Mediterranean Restaurant
14,Clinton,Theater,Park,Coffee Shop,Mediterranean Restaurant,Pizza Place,Gym / Fitness Center,Gym,Concert Hall,Performing Arts Venue,Indie Theater
15,Midtown,Theater,Hotel,Gym,Concert Hall,Sandwich Place,Park,Boxing Gym,Burger Joint,Museum,Plaza
16,Murray Hill,Theater,Hotel,American Restaurant,Gourmet Shop,Boxing Gym,Gym / Fitness Center,Park,Burger Joint,Sandwich Place,Pizza Place
33,Midtown South,Theater,Gym / Fitness Center,Gourmet Shop,Park,American Restaurant,Miscellaneous Shop,Beer Bar,Japanese Restaurant,Indie Theater,Coffee Shop
39,Hudson Yards,Theater,Park,Art Gallery,Coffee Shop,Bar,Pizza Place,Gym,Gym / Fitness Center,Bakery,American Restaurant


### Cluster 2

In [54]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 1, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Chinatown,Hotel,Sandwich Place,Pizza Place,Bakery,Thai Restaurant,Italian Restaurant,Spa,Yoga Studio,Gym / Fitness Center,Sushi Restaurant
8,Upper East Side,Art Museum,Exhibit,Bakery,Italian Restaurant,Plaza,Bar,Thai Restaurant,Yoga Studio,Playground,Sushi Restaurant
9,Yorkville,Italian Restaurant,Coffee Shop,Park,Bakery,Thai Restaurant,Ice Cream Shop,American Restaurant,Gym,Mexican Restaurant,Pizza Place
10,Lenox Hill,Bakery,Italian Restaurant,Park,Art Museum,Thai Restaurant,Bar,Sushi Restaurant,Ice Cream Shop,Cycle Studio,Grocery Store
11,Roosevelt Island,Park,Italian Restaurant,Gym / Fitness Center,Thai Restaurant,Art Museum,Coffee Shop,Cocktail Bar,Gym,Sushi Restaurant,Brewery
12,Upper West Side,Park,Bakery,Italian Restaurant,Mediterranean Restaurant,Ice Cream Shop,Sushi Restaurant,Coffee Shop,Gym,Chinese Restaurant,Playground
17,Chelsea,Park,Gym / Fitness Center,Bakery,Yoga Studio,Hotel,Coffee Shop,Jazz Club,Gourmet Shop,Beer Bar,New American Restaurant
18,Greenwich Village,Pizza Place,Italian Restaurant,New American Restaurant,French Restaurant,Garden,Grocery Store,Gym / Fitness Center,Gourmet Shop,Bakery,Salad Place
19,East Village,Pizza Place,Ice Cream Shop,Gourmet Shop,Bakery,Café,Park,Wine Shop,Wine Bar,Speakeasy,Hotel
20,Lower East Side,Cocktail Bar,Coffee Shop,Wine Bar,Sandwich Place,Park,Juice Bar,Ice Cream Shop,Pizza Place,Hotel,Thai Restaurant


### Cluster 3

In [55]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 2, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
7,East Harlem,Park,Café,Coffee Shop,Pizza Place,Gym,Plaza,Italian Restaurant,Fountain,Cocktail Bar,Thai Restaurant
21,Tribeca,Park,Italian Restaurant,Coffee Shop,Sushi Restaurant,Cycle Studio,Memorial Site,Café,Hotel,Gym / Fitness Center,Grocery Store
25,Manhattan Valley,Park,Coffee Shop,Grocery Store,Seafood Restaurant,Playground,Scenic Lookout,Italian Restaurant,Fountain,Café,Indian Restaurant
28,Battery Park City,Park,Coffee Shop,Gym,Hotel,Italian Restaurant,Memorial Site,Plaza,Gym / Fitness Center,Steakhouse,Burger Joint
29,Financial District,Park,Coffee Shop,Gym / Fitness Center,Gym,Pizza Place,Plaza,Cocktail Bar,Hotel,Memorial Site,Burger Joint
32,Civic Center,Park,Coffee Shop,Hotel,Italian Restaurant,Bakery,Memorial Site,Café,Dance Studio,Falafel Restaurant,Thai Restaurant


### Cluster 4

In [56]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 3, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Hamilton Heights,Coffee Shop,Park,Bar,Italian Restaurant,Yoga Studio,French Restaurant,Cocktail Bar,Tennis Court,Jazz Club,Juice Bar
5,Manhattanville,Park,Coffee Shop,Yoga Studio,Italian Restaurant,Wine Shop,American Restaurant,Cocktail Bar,French Restaurant,Pizza Place,Burger Joint
6,Central Harlem,Coffee Shop,Bar,Park,Cocktail Bar,Southern / Soul Food Restaurant,Yoga Studio,American Restaurant,Theater,Wine Shop,Burger Joint
26,Morningside Heights,Park,Coffee Shop,Italian Restaurant,American Restaurant,Playground,Grocery Store,Seafood Restaurant,Pizza Place,Farmers Market,Bar


### Cluster 5

In [57]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 3, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Hamilton Heights,Coffee Shop,Park,Bar,Italian Restaurant,Yoga Studio,French Restaurant,Cocktail Bar,Tennis Court,Jazz Club,Juice Bar
5,Manhattanville,Park,Coffee Shop,Yoga Studio,Italian Restaurant,Wine Shop,American Restaurant,Cocktail Bar,French Restaurant,Pizza Place,Burger Joint
6,Central Harlem,Coffee Shop,Bar,Park,Cocktail Bar,Southern / Soul Food Restaurant,Yoga Studio,American Restaurant,Theater,Wine Shop,Burger Joint
26,Morningside Heights,Park,Coffee Shop,Italian Restaurant,American Restaurant,Playground,Grocery Store,Seafood Restaurant,Pizza Place,Farmers Market,Bar
